# Grad-CAM (Gradient-weighted Class Activation Mapping) 教學筆記

## 1. 什麼是 Grad-CAM?

Grad-CAM (Gradient-weighted Class Activation Mapping) 是一種用於解釋深度學習模型決策過程的可視化技術。它專注於卷積神經網絡（CNN），透過可視化模型對輸入圖像不同區域的關注點，幫助我們理解模型如何得出特定的預測。Grad-CAM 生成的“熱圖”會將模型最關注的區域高亮顯示，讓我們能夠直觀地看到模型在哪些區域做出了決策。

## 2. Grad-CAM 的工作原理

Grad-CAM 是基於 CNN 的特徵圖（feature maps）和梯度信息進行計算。具體來說，它會通過以下幾個步驟來生成可視化熱圖：

1. **前向傳播：** 輸入圖像傳遞給模型，經過多層卷積層和池化層，生成特徵圖。
2. **反向傳播：** 對於模型的輸出結果，計算相對於特徵圖的梯度。這些梯度用來衡量每個卷積層的輸出對於最終決策的重要性。
3. **權重計算：** 這些梯度被用作權重，與每個卷積層的特徵圖進行加權平均，生成“熱圖”。
4. **生成熱圖：** 熱圖展示了模型對每個圖像區域的關注程度。顏色越熱的區域代表模型對該區域的關注度越高。

## 3. 為什麼使用 Grad-CAM？

Grad-CAM 的優勢在於它不需要修改原始模型結構即可應用在大多數 CNN 模型上。這讓它成為一個強大且靈活的工具，能夠被用來診斷模型錯誤、理解模型行為，並且可以幫助解釋物件檢測模型（例如 YOLO）為什麼會在某些區域做出檢測。

## 4. Grad-CAM 的應用場景

- **醫療影像：** Grad-CAM 可以幫助醫生理解 AI 模型如何對醫療圖像（例如 X 光、CT 圖像）進行診斷。
- **物件檢測：** 在 YOLO 模型中，Grad-CAM 可以展示模型在進行物件檢測時，對哪些區域做出了重點判斷。
- **自動駕駛：** 解釋自動駕駛系統在分析路面場景時，重點關注了哪些區域，進一步提升其安全性。

## 5. Grad-CAM 的實作步驟

### 1. 取得 CNN 模型的輸出
首先，我們需要獲取模型的卷積層輸出與最終分類結果，這是進行 Grad-CAM 計算的基礎。

### 2. 計算對特徵圖的梯度
通過反向傳播，我們可以計算分類結果對於最後一層卷積層特徵圖的梯度，這些梯度反映了每個特徵圖對最終決策的貢獻度。

### 3. 加權特徵圖
利用這些梯度對特徵圖進行加權，這樣能夠強調對最終分類結果影響最大的區域。

### 4. 生成熱圖
將加權的特徵圖進行聚合，並將結果可視化為一個熱圖。

## 6. 結論

Grad-CAM 是一個強大的工具，能夠幫助我們解釋深度學習模型的決策過程，特別是對於卷積神經網絡的可視化應用，它可以提供關於模型內部運作的重要洞察。無論是應用於醫療影像、物件檢測，還是其他深度學習領域，Grad-CAM 都能顯著提升模型解釋的透明度與可信度。

---

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image

# 加載你訓練好的 YOLO 模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path='D:/Learning_Python/30-Day_AI_Deep_Learning_Plan/yolov5-master/runs/train/exp2/weights/best.pt')
model.eval()    # 設定為評估模式

# 取得某一張圖片
img = Image.open('D:\Learning_Python\30-Day_AI_Deep_Learning_Plan\第3週：物件檢測與 YOLO\Day17\images\train\000851.jpg')  # 讀取圖片
img = np.array(img)  # PIL Image 轉換為 NumPy 陣列，因 YOLO 模型使用的是 NumPy 格式

# 進行推理
results = model(img)  # 輸入圖片到 YOLO 模型進行預測

# 取得類別分數最大的索引（此步驟是以物件偵測為基礎）
pred_class = results.names[results.pred[0][:, -1].int().item()] # YOLO 模型會返回物件名稱

# 產生 Grad-CAM
gradients = model.get_gradients()   # 取得梯度
features = model.get_features() # 取得卷積層輸出

# 計算加權平均，生成 Grad-CAM
weights = torch.mean(gradients, dim=(2, 3))  # 平均化梯度
grad_cam = torch.zeros(features.shape[2:], dtype=torch.float32) # 初始化 Grad-CAM

# 加權平均
for i, w in enumerate(weights):
    grad_cam += w * features[0, i, :, :]    

grad_cam = torch.relu(grad_cam)  # ReLU 使負數變為 0

# 轉換為可視化格式
grad_cam = grad_cam.detach().numpy()    # 轉換為 NumPy 陣列
grad_cam = cv2.resize(grad_cam, (img.shape[1], img.shape[0]))   # 調整大小
grad_cam = (grad_cam - grad_cam.min()) / (grad_cam.max() - grad_cam.min())  # 正規化
grad_cam = np.uint8(255 * grad_cam) # 轉換為 0-255 的整數

# 將 Grad-CAM 熱圖疊加到原始圖像上
heatmap = cv2.applyColorMap(grad_cam, cv2.COLORMAP_JET)   # 生成熱圖
superimposed_img = heatmap * 0.4 + img    # 疊加熱圖

# 顯示圖像
cv2.imshow('Grad-CAM', superimposed_img)
cv2.waitKey(0)